In [1]:
data_location = "/mnt/NewHDD"

In [14]:
import tensorflow as tf
import numpy as np
import InputList
import BinaryReader
import TFRecordsHandler
import time
import logging
import os
from importlib import reload
reload(TFRecordsHandler)
logging.basicConfig(filename='benchmark.log', encoding='utf-8', level=logging.DEBUG)

In [174]:
logging.info("Benchmark Start")

In [3]:
br = BinaryReader.BinaryReader()
bag = br.create_test_dataset([InputList.diabetic_testing_files[0]])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-02-22 17:16:50.811137: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-22 17:16:50.811574: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-22 17:16:50.811582: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-22 17:16:50.811592: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (zeissstudent1): /proc/driver/nvidia/version does not exist
2022-02-22 17:16:50.812232: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild Ten

In [4]:
def timeing(func):
    def inner(*args, **kwargs):
        start = time.time()
        logging.info(f"Starting with {func.__name__}")
        func(*args, **kwargs)
        end = time.time()
        logging.info(f"Finished with {func.__name__} in {end - start}")
    return inner

def instance_to_file(tensor, filename):
    with open(filename, 'wb') as f:
        np.save(f, tensor[0].numpy())

def file_to_instance(file):
    with open(file, 'rb') as f:
        a = np.load(f)
    return a, int(os.path.basename(file).decode('ascii')[0])

def numpy_instance_generator(instance_file_list):
    for file in instance_file_list:
        yield file_to_instance(file)

In [ ]:
def tfrecords_real_writer():
    br = BinaryReader.BinaryReader()
    for file in file_list:
        bag = br.create_test_dataset([file])
        i = 0
        dirname = os.path.join(data_location, "tfrecords")
        if not os.path.exists(dirname):
            os.mkdir(dirname)
        for instance in bag:
            filename = os.path.join(dirname, f"{os.path.basename(file[0])}_{i}")
            TFRecordsHandler.write_images_to_tfr_short(*instance, filename=filename)
            i += 1

('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D39/links',
 'raw_1536x2048x2045x2_31711.bin')

In [17]:
@timeing
def tfrecords_writer():
    i = 0
    dirname = os.path.join(data_location, "tfrecords")
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    for instance in bag:
        filename = os.path.join(dirname, f"file_{i}")
        TFRecordsHandler.write_images_to_tfr_short(*instance, filename=filename)
        i += 1

@timeing
def numpy_writer():
    i = 0
    dirname = os.path.join(data_location, "numpy")
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    for instance in bag:
        filename = os.path.join(dirname, f"{instance[1]}_file_{i}.npy")
        instance_to_file(instance, filename)
        i += 1

def tfrecords_reader():
    dir_location = os.path.join(data_location, "tfrecords")
    list_of_files = os.listdir(dir_location)
    list_of_file_paths = list(map(lambda x: os.path.join(dir_location, x), list_of_files))
    dataset = TFRecordsHandler.get_dataset_small(list_of_file_paths)
    return dataset

def numpy_reader():
    dir_location = os.path.join(data_location, "numpy")
    list_of_files = os.listdir(dir_location)
    list_of_file_paths = list(map(lambda x: os.path.join(dir_location, x), list_of_files))
    dataset = tf.data.Dataset.from_tensor_slices(list_of_file_paths) \
                .interleave(lambda x:
        tf.data.Dataset.from_generator(numpy_instance_generator, args=[list_of_file_paths],
                                             output_signature=(
                                                tf.TensorSpec(
                                                    shape=(1536, 102, 102, 1),
                                                    dtype=np.dtype('u2')),
                                                tf.TensorSpec(
                                                    shape=(),
                                                    dtype=np.dtype('u1')))
                                             ),
                            num_parallel_calls=tf.data.AUTOTUNE,
                            deterministic=True)\
        .prefetch(tf.data.AUTOTUNE)
    return dataset

def classy_reader():
    return br.create_dataset([InputList.diabetic_testing_files[0]], True)

@timeing
def tfrecords_iterator(dataset):
    for sample in dataset:
        garbage = sample[0]
    print(garbage)

@timeing
def numpy_iterator(dataset):
    garbage = 0
    for sample in dataset:
        garbage += sample[1]
    print(garbage)

@timeing
def classy_iterator(dataset):
    garbage = 0
    for sample in dataset:
        garbage += sample[1]
    print(garbage)

In [18]:
tfrecords_writer()
#numpy_writer()

In [8]:
tfdataset = tfrecords_reader()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [187]:
numpy_dataset = numpy_reader()

In [160]:
classy_dataset = classy_reader()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 85)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 85)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [13]:
tfrecords_iterator(tfdataset)

tf.Tensor(
[[[[38176]
   [34560]
   [31648]
   ...
   [36640]
   [37088]
   [32384]]

  [[34432]
   [32768]
   [33632]
   ...
   [35616]
   [33376]
   [35616]]

  [[37216]
   [36064]
   [33920]
   ...
   [34816]
   [32416]
   [31456]]

  ...

  [[34752]
   [32224]
   [31168]
   ...
   [33888]
   [34304]
   [33728]]

  [[34272]
   [36896]
   [31232]
   ...
   [33568]
   [39104]
   [33344]]

  [[34144]
   [33088]
   [34528]
   ...
   [35040]
   [34080]
   [36160]]]


 [[[34336]
   [32640]
   [30976]
   ...
   [35200]
   [36224]
   [33024]]

  [[37696]
   [34400]
   [31904]
   ...
   [34720]
   [34912]
   [30784]]

  [[37472]
   [35232]
   [33184]
   ...
   [34240]
   [34816]
   [34816]]

  ...

  [[37600]
   [33664]
   [28288]
   ...
   [35296]
   [34944]
   [31776]]

  [[37216]
   [36416]
   [29440]
   ...
   [33504]
   [35584]
   [34624]]

  [[33760]
   [34016]
   [32096]
   ...
   [33984]
   [35648]
   [33440]]]


 [[[34368]
   [32256]
   [32192]
   ...
   [36320]
   [36800]
   [34624

In [188]:
numpy_iterator(numpy_dataset)

KeyboardInterrupt: 

In [183]:
classy_iterator(classy_dataset)

tf.Tensor(135, shape=(), dtype=uint8)


In [ ]:
parsed_dataset = tf.data.TFRecordDataset(["images.tfrecords"])

In [ ]:
for parsed_record in parsed_dataset.take(10):
  print(repr(parsed_record))

In [ ]:
from TFRecordsHandler import get_dataset_small

In [ ]:
instance_to_file(sample)

In [ ]:
file_list = ["0_file.npy"]
dataset = tf.data.Dataset.from_generator(numpy_instance_generator, args=[file_list],
                                         output_signature=(
                                            tf.TensorSpec(
                                                shape=(1536, 102, 102, 1),
                                                dtype=np.dtype('u2')),
                                            tf.TensorSpec(
                                                shape=(),
                                                dtype=np.dtype('u1')))
                                         )

In [ ]:
dataset